<a href="https://colab.research.google.com/github/Muthukuda/dlaicourse/blob/master/MIRNetv2_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Enriched Features for Fast Image Restoration and Enhancement (TPAMI 2022) [![paper](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://www.waqaszamir.com/publication/zamir-2022-mirnetv2/) 

<hr />

This is a demo to run MIRNet_v2 on you own images for the following tasks
- Real Image Denoising
- Super-Resolution
- Contrast Enhancement
- Low-Light Image Enhancement

# 1. Setup
- First, in the **Runtime** menu -> **Change runtime type**, make sure to have ```Hardware Accelerator = GPU```
- Clone repo and install dependencies. 


In [ ]:
import os

if os.path.isdir('MIRNetv2'):
  !rm -r MIRNetv2

# Clone MIRNetv2
!git clone https://github.com/swz30/MIRNetv2.git
%cd MIRNetv2

# 2. Define Task and Download Pre-trained Models
Uncomment the task you would like to perform

In [ ]:
# task = 'real_denoising'
# task = 'super_resolution'
# task = 'contrast_enhancement'
task = 'lowlight_enhancement'

# Download the pre-trained models
if task is 'real_denoising':
  !wget https://github.com/swz30/MIRNetv2/releases/download/v1.0.0/real_denoising.pth -P Real_Denoising/pretrained_models
if task is 'super_resolution':
  !wget https://github.com/swz30/MIRNetv2/releases/download/v1.0.0/sr_x4.pth -P Super_Resolution/pretrained_models
if task is 'contrast_enhancement':
  !wget https://github.com/swz30/MIRNetv2/releases/download/v1.0.0/enhancement_fivek.pth -P Enhancement/pretrained_models
if task is 'lowlight_enhancement':
  !wget https://github.com/swz30/MIRNetv2/releases/download/v1.0.0/enhancement_lol.pth -P Enhancement/pretrained_models


# 3. Upload Images
Either download the sample images or upload your own images

In [ ]:
import os
import shutil
from google.colab import files

# Download sample images
!rm -r demo/*
!wget https://github.com/swz30/MIRNetv2/releases/download/v1.0.0/sample_images.zip -P demo
shutil.unpack_archive('demo/sample_images.zip', 'demo/')
os.remove('demo/sample_images.zip')

# OR Uncomment the following block if you would like to upload your own images. 
"""
!rm -r demo/*
input_dir = 'demo/sample_images/'+task+'/degraded'
os.makedirs(input_dir, exist_ok=True)
uploaded = files.upload()
for filename in uploaded.keys():
  input_path = os.path.join(input_dir, filename)
  shutil.move(filename, input_path)
"""

# 4. Prepare Model and Load Checkpoint

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np

def get_weights_and_parameters(task, parameters):
    if task == 'real_denoising':
        weights = os.path.join('Real_Denoising', 'pretrained_models', 'real_denoising.pth')
    elif task == 'super_resolution':
        weights = os.path.join('Super_Resolution', 'pretrained_models', 'sr_x4.pth')
        parameters['scale'] =  4
    elif task == 'contrast_enhancement':
        weights = os.path.join('Enhancement', 'pretrained_models', 'enhancement_fivek.pth')
    elif task == 'lowlight_enhancement':
        weights = os.path.join('Enhancement', 'pretrained_models', 'enhancement_lol.pth')
    return weights, parameters


# Get model weights and parameters
parameters = {
    'inp_channels':3,
    'out_channels':3, 
    'n_feat':80,
    'chan_factor':1.5,
    'n_RRG':4,
    'n_MRB':2,
    'height':3,
    'width':2,
    'bias':False,
    'scale':1,
    'task': task
    }

weights, parameters = get_weights_and_parameters(task, parameters)

load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'mirnet_v2_arch.py'))
model = load_arch['MIRNet_v2'](**parameters)
model.cuda()

checkpoint = torch.load(weights)
model.load_state_dict(checkpoint['params'])
model.eval()


# 5. Inference

In [ ]:
input_dir = 'demo/sample_images/'+task+'/degraded'
out_dir = 'demo/sample_images/'+task+'/restored'
os.makedirs(out_dir, exist_ok=True)
extensions = ['jpg', 'JPG', 'png', 'PNG', 'jpeg', 'JPEG', 'bmp', 'BMP']
files = natsorted(glob(os.path.join(input_dir, '*')))

img_multiple_of = 4

print(f"\n ==> Running {task} with weights {weights}\n ")
with torch.no_grad():
  for filepath in tqdm(files):
      # print(file_)
      torch.cuda.ipc_collect()
      torch.cuda.empty_cache()
      img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
      input_ = torch.from_numpy(img).float().div(255.).permute(2,0,1).unsqueeze(0).cuda()

      # Pad the input if not_multiple_of 4
      h,w = input_.shape[2], input_.shape[3]
      H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
      padh = H-h if h%img_multiple_of!=0 else 0
      padw = W-w if w%img_multiple_of!=0 else 0
      input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

      restored = model(input_)
      restored = torch.clamp(restored, 0, 1)

      # Unpad the output
      restored = restored[:,:,:h,:w]

      restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
      restored = img_as_ubyte(restored[0])

      filename = os.path.split(filepath)[-1]
      cv2.imwrite(os.path.join(out_dir, filename),cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))

# 6. Visualize Results


In [ ]:
import matplotlib.pyplot as plt
inp_filenames = natsorted(glob(os.path.join(input_dir, '*')))
out_filenames = natsorted(glob(os.path.join(out_dir, '*')))

## Will display only first 5 images
num_display_images = 5
if len(inp_filenames)>num_display_images:
  inp_filenames = inp_filenames[:num_display_images]
  out_filenames = out_filenames[:num_display_images]

print(f"Results: {task}")
for inp_file, out_file in zip(inp_filenames, out_filenames):
  degraded = cv2.cvtColor(cv2.imread(inp_file), cv2.COLOR_BGR2RGB)
  restored = cv2.cvtColor(cv2.imread(out_file), cv2.COLOR_BGR2RGB)
  ## Display Images
  fig, axes = plt.subplots(nrows=1, ncols=2)
  dpi = fig.get_dpi()
  fig.set_size_inches(900/ dpi, 448 / dpi)
  plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
  axes[0].axis('off')
  axes[0].imshow(degraded)
  axes[1].axis('off')
  axes[1].imshow(restored)
  plt.show()

# 7. Download Results

In [ ]:
from google.colab import files
zip_filename = f"Restormer_{task}.zip"
os.system(f"zip -r {zip_filename} demo/sample_images/{task}")
files.download(zip_filename)